**Provide a comparative analysis of the following different order execution types:**
- Market Order,
- Limit Order,
- Stop Limit Order,
- Stop Market Order

Clearly state the relative merits and demerits of each.
- How will expanding and shrinking Bid-Ask spreads affect the efficiency of execution of each of
the above Order Types.

** Develop a simple model to estimate execution slippage of each of the order types based on latest available values of the 9 main data points:**
- Open
- High
- Low
- Close
- Volume
- Best Bid
- Best Ask
- Highest Bid Quantity
- Highest Ask Quantity

In Other words, the model should provide a realistic analysis of how much slippage we can expect in an order given the latest available values of the 9 data points.

Write a simple Python program that implements your model. The program should be able print out the expected slippages in the different order types based on values of 9 data points (Do not ask the user to input these values manually, rather define them as variables in your python file with proper commenting).

In [7]:
import numpy as np
import pandas as pd
from pandas_datareader import data, wb
import matplotlib.pyplot as plt
import talib as ta
import datetime

## 1. Download 15 yrs DJIA
start = datetime.datetime(2002, 6, 1)
end = datetime.datetime(2017, 6, 1)

spy = data.DataReader('SPY',  'google', start, end)
#data = data['Close']
#returns = np.log(data / data.shift(1))

###

#spy = spy.drop('Volume', axis=1)
spy = pd.DataFrame(spy)
spy.head()

,Open,High,Low,Close,Volume
Date,,,,,
2002-06-03,107.09,107.60,104.13,104.37,25221400
2002-06-04,104.15,105.20,103.28,104.63,25639300
2002-06-05,104.95,105.67,104.35,105.61,19599600
2002-06-06,105.54,105.60,103.15,103.46,21011100
2002-06-07,101.79,103.92,101.72,103.34,23357500


In [24]:
import random
spy['Bbid'] = spy['Close'] * 0.98
spy['Bask'] = spy['Close'] * 1.02 
spy['sprd'] = spy['Bask'] - spy['Bbid'] 
spy['vol']  = spy['High'] - spy['Close']
spy.head()

,Open,High,Low,Close,Volume,Bbid,Bask,sprd,vol
Date,,,,,,,,,
2002-06-03,107.09,107.60,104.13,104.37,25221400,102.2826,106.4574,4.1748,3.23
2002-06-04,104.15,105.20,103.28,104.63,25639300,102.5374,106.7226,4.1852,0.57
2002-06-05,104.95,105.67,104.35,105.61,19599600,103.4978,107.7222,4.2244,0.06
2002-06-06,105.54,105.60,103.15,103.46,21011100,101.3908,105.5292,4.1384,2.14
2002-06-07,101.79,103.92,101.72,103.34,23357500,101.2732,105.4068,4.1336,0.58


- Your trading volume: The more shares you want to buy, the longer the order execution takes. 
- The longer the order execution takes, the further the fill price might be.
- The bid-ask spread: This is the difference in the price quoted for an immediate buy (ask) and the price quoted for an immediate sale (bid). To get an order filled, you usually have to cross the spread. This is typically on the far side for you. - - If you want to sell 100 shares of Stock X you need to find a buyer for them. 
- If there is one in the orderbook you will find him at the other side of the spread. Large bid/ask spreads lead to a high slippage.
- The volatility of the market: For the sake of simplicity, let’s define volatility as the average change of price per unit of time. 
- Thus, if the volatility is high, it’s evident that slippage will be higher in volatile markets since prices tend to move more while your order is executed

In [26]:
#slippage percent
vol = spy['vol']
x = vol
#slppct = x

#highest Bid quantity is a function of Volatility, depth of book
#to emulate depth of book I assume 1/2 total daily volume is single day depth of book
#to emulate less depth with more volatilty
vlm = spy['Volume']
spy['HBQ'] = ( vlm / 2 ) / x
spy['HAQ'] = ( vlm / 2 ) / x
spy.head()

,Open,High,Low,Close,Volume,Bbid,Bask,sprd,vol,HBQ,HAQ
Date,,,,,,,,,,,
2002-06-03,107.09,107.60,104.13,104.37,25221400,102.2826,106.4574,4.1748,3.23,3.904241e+06,3.904241e+06
2002-06-04,104.15,105.20,103.28,104.63,25639300,102.5374,106.7226,4.1852,0.57,2.249061e+07,2.249061e+07
2002-06-05,104.95,105.67,104.35,105.61,19599600,103.4978,107.7222,4.2244,0.06,1.633300e+08,1.633300e+08
2002-06-06,105.54,105.60,103.15,103.46,21011100,101.3908,105.5292,4.1384,2.14,4.909136e+06,4.909136e+06
2002-06-07,101.79,103.92,101.72,103.34,23357500,101.2732,105.4068,4.1336,0.58,2.013578e+07,2.013578e+07


In [31]:
#slippage factor
hbq = spy['HBQ']
haq = spy['HAQ']

#bestbidask
bb = spy['Bbid'] 
ba = spy['Bask'] 

#HighestHigh
HH = spy['High']

#LowestLow 
LL = spy['Low']

#slpfctr Buying
spy['slpfctrB'] = HH - (bb * (1 + hbq)) 

#slpfctr selling
spy['slpfctrA'] = LL + (ba * (1 - haq))

#true depth / bid ask considering slippage factor
spy['truB'] = spy['Bbid'] - spy['slpfctrB']
spy['trua'] = spy['Bask'] + spy['slpfctrA']
spy.head()

,Open,High,Low,Close,Volume,Bbid,Bask,sprd,vol,HBQ,HAQ,slpfctrB,slpfctrA,truB,trua
Date,,,,,,,,,,,,,,,
2002-06-03,107.09,107.60,104.13,104.37,25221400,102.2826,106.4574,4.1748,3.23,3.904241e+06,3.904241e+06,-3.993360e+08,-4.156352e+08,3.993361e+08,-4.156351e+08
2002-06-04,104.15,105.20,103.28,104.63,25639300,102.5374,106.7226,4.1852,0.57,2.249061e+07,2.249061e+07,-2.306129e+09,-2.400257e+09,2.306129e+09,-2.400256e+09
2002-06-05,104.95,105.67,104.35,105.61,19599600,103.4978,107.7222,4.2244,0.06,1.633300e+08,1.633300e+08,-1.690430e+10,-1.759427e+10,1.690430e+10,-1.759427e+10
2002-06-06,105.54,105.60,103.15,103.46,21011100,101.3908,105.5292,4.1384,2.14,4.909136e+06,4.909136e+06,-4.977412e+08,-5.180569e+08,4.977413e+08,-5.180568e+08
2002-06-07,101.79,103.92,101.72,103.34,23357500,101.2732,105.4068,4.1336,0.58,2.013578e+07,2.013578e+07,-2.039214e+09,-2.122447e+09,2.039215e+09,-2.122447e+09
